In [ ]:
from pprint import pprint
import datetime
import auxiliary_functions

In [ ]:
%run /Users/michel.davidovich@tomtom.com/STAN-112/auxiliary_notebooks/0.0-useful_functions

Importing the compose_depth_levels_to_providers function...

Importing the Bootstrapping functions: "bootstrap_resample" and "percentile_bootstrap"...

Importing the add_to_end function...

Adding Errors for handling...

Importing function to remove underscores at the beginning of suffixes and end_of_strings...

In [ ]:
%run /Users/michel.davidovich@tomtom.com/STAN-112/auxiliary_notebooks/0.1-API_calls_imports

/databricks/python/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
 warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')

In [ ]:
%run /Users/michel.davidovich@tomtom.com/STAN-207/auxiliary_functions/0.0_preprocess_ADX_search_logs_insights_data

Requirement already satisfied: pycountry in /databricks/python3/lib/python3.8/site-packages (22.3.5)
Requirement already satisfied: setuptools in /usr/local/lib/python3.8/dist-packages (from pycountry) (52.0.0)
 WARNING: You are using pip version 21.0.1; however, version 22.2.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: country_list in /databricks/python3/lib/python3.8/site-packages (1.0.0)
 WARNING: You are using pip version 21.0.1; however, version 22.2.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: country_converter in /databricks/python3/lib/python3.8/site-packages (0.7.7)
Requirement already satisfied: pandas>=1.0 in /databricks/python3/lib/python3.8/site-packages (from country_converter) (1.2.4)
Requirement already satisfied: pytz>=2017.3 in /databricks/python3/lib/python3.8/site-packages (from pandas>=1.0->country_converter) (2020.5)
Requirement already satisfied: numpy>=1.16.5 in /databricks/python3/lib/python3.8/site-packages (from pandas>=1.0->country_converter) (1.19.2)
Requirement already satisfied: python-dateutil>=2.7.3 in /databricks/python3/lib/python3.8/site-packages (from pandas>=1.0->country_converter) (2.8.1)
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.8/site-packages (from python-dateutil>=2.7.3->pandas>=1.0->country_converter) (1.15.0)
 WARNING: You are using pip version 21.0.1; however, version 22.2.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [ ]:
%run /Users/michel.davidovich@tomtom.com/STAN-207/2.0-parse_API_calls

# Defining the api calls process:

In [ ]:
def apply_tt_api(x: pd.Series, api_instance, limit: int = 5, sleep: int = 1, debug: bool = False) -> dict:
    """Function that gets a row from a dataframe and makes the geocoding API calls using the searched_query to the TT API.
    
    :param x: Row of the DataFrame on which you want to perform API calls. The DataFrame must at least contain
    :type x: pd.Series
    :param api_instance: Instantiation of the API class you want to run the results on. Generally, api_instance = TomtomApi().
    :type api_instance: __main__.TomtomApi()
    :param limit: Max number of responses you want from the TT API. Defaults to 5
    :type limit: int, optional
    :param sleep: Number of seconds of pause between two API calls, defaults to 1 second.
    :type sleep: int, optional
    :param debug: Define whether you want to output the progression through the different searched queries. Defaults to False, which doesn't print the searched query
    :type debug: bool, optional
    :return: A dictionary with the full TT API geocoding response to the searched query.
    :rtype: dict
    """
    searched_query = x['searched_query']
    country = x['country']
    
    if debug:
        print(f'collected searched_query: {searched_query}')
        
    return api_instance.call_api({'address': searched_query, 'sleep': sleep, 'limit': limit, 'country': country}, call_type='geocode')

In [ ]:
def full_function_make_api_calls(
    path_to_read: str, path_to_save: str, country_iso2: str, limit: int = 5, sleep: int = 1, threshold: float = 0.85, test_size: int or None = None
) -> pd.DataFrame:
    """Function that makes the calls to the TomTom API after reading the Data from a path in the DBFS. You can pass an argument to save the api responses to another file in the DBFS as well.

    :param path_to_read: Path where you want to read the search logs from.
    :type path_to_read: str
    :param path_to_save: Path where you want to save the data, if any. Defaults to None, which means you don't need to save the results.
    :type path_to_save: str or None
    :param country_iso2: Country in ISO-2 format.
    :type country_iso2: str
    :param limit: Max number of responses you want from the TT API, defaults to 5
    :type limit: int, optional
    :param sleep: Time to wait between queries to the API server, defaults to 1
    :type sleep: int, optional
    :param threshold: The minimum level of confidence that is considered a match from the API response, defaults to 0.85, which means the confidence that the API has on what it is responding for the address to be a match is at least 85%.
    :type threshold: float, optional
    :param test_size: Size of the sample you want to take from the read dataframe. This parameter should only be used for testing purposes! It should be left at None! Defaults to None.
    :type test_size: int or None
    :return: A dataframe that contains the parsed response for each of the addresses returned. It contains the coordinates, the searched query, the confidence level, the type of query, the bounding box (if it is a geometry) and whether it is considered a match or not under the threshold criteria.
    :rtype: pd.DataFrame
    """
    api = TomtomApi()
    if ((test_size is None) or (test_size == 0)):
        df = pd.read_parquet(path_to_read).reset_index(drop=True)
    else:
        df = pd.read_parquet(path_to_read).reset_index(drop=True).sample(test_size, random_state=101)
        
    df = df[df['method_name'] != 'search 2 nearbySearch']
    
    function = lambda x: api_calls_parser((apply_tt_api(x, api, limit=limit, sleep=sleep)), x.name, x['searched_query'], threshold)
    
    result =  df.apply(function, axis=1)
    
    final_df = pd.DataFrame(result.tolist())
    
    if path_to_save not in ['', None]:
        final_df.to_parquet(path_to_save)
        
    return final_df

In [ ]:
country_iso2 = 'PT'
interest_date = '19-09-2022'

In [ ]:
non_comp_path = f'/dbfs/FileStore/STAN-207/non_complete_responses_{interest_date}_{country_iso2}.parquet'
comp_path     = f'/dbfs/FileStore/STAN-207/complete_responses_{interest_date}_{country_iso2}.parquet'

today = str(datetime.datetime.today().strftime('%d-%m-%Y'))

non_comp_save_path1 = f'/dbfs/FileStore/STAN-207/parsed_responses/incomplete_{today}_{country_iso2}_v1.parquet'
non_comp_save_path2 = f'/dbfs/FileStore/STAN-207/parsed_responses/incomplete_{today}_{country_iso2}_v2.parquet'
comp_save_path     = f'/dbfs/FileStore/STAN-207/parsed_responses/complete_{today}_{country_iso2}.parquet'

In [ ]:
pd.read_parquet(comp_path).shape

Out[35]: (2059, 14)

In [ ]:
today

Out[39]: '20-09-2022'

In [ ]:
#result_comp     = full_function_make_api_calls(comp_path, comp_save_path, country_iso2, 1, 1.5, 0.85)
result_non_comp = full_function_make_api_calls(non_comp_path, non_comp_save_path, country_iso2, 1, 1.5, 0.85)

--------------------------------------------------------------------------- 
 HTTPError Traceback (most recent call last)
 <command-3167774330271464> in <module> 
 1 #result_comp = full_function_make_api_calls(comp_path, comp_save_path, country_iso2, 1, 1.5, 0.85) 
 ----> 2 result_non_comp = full_function_make_api_calls ( non_comp_path , non_comp_save_path , country_iso2 , 1 , 1.5 , 0.85 ) 

 <command-1287881767488839> in full_function_make_api_calls (path_to_read, path_to_save, country_iso2, limit, sleep, threshold, test_size) 
 31 function = lambda x : api_calls_parser ( ( apply_tt_api ( x , api , limit = limit , sleep = sleep ) ) , x . name , x [ 'searched_query' ] , threshold ) 
 32 
 ---> 33 result = df . apply ( function , axis = 1 ) 
 34 
 35 final_df = pd . DataFrame ( result . tolist ( ) ) 

 /databricks/python/lib/python3.8/site-packages/pandas/core/frame.py in apply (self, func, axis, raw, result_type, args, **kwds) 
 7766 kwds = kwds , 
 7767 )
 -> 7768 return op . get_result ( ) 
 7769 
 7770 def applymap ( self , func , na_action : Optional [ str ] = None ) -> DataFrame : 

 /databricks/python/lib/python3.8/site-packages/pandas/core/apply.py in get_result (self) 
 183 return self . apply_raw ( ) 
 184 
 --> 185 return self . apply_standard ( ) 
 186 
 187 def apply_empty_result ( self ) : 

 /databricks/python/lib/python3.8/site-packages/pandas/core/apply.py in apply_standard (self) 
 274 
 275 def apply_standard ( self ) : 
 --> 276 results , res_index = self . apply_series_generator ( ) 
 277 
 278 # wrap results 

 /databricks/python/lib/python3.8/site-packages/pandas/core/apply.py in apply_series_generator (self) 
 288 for i , v in enumerate ( series_gen ) : 
 289 # ignore SettingWithCopy here in case the user mutates 
 --> 290 results [ i ] = self . f ( v ) 
 291 if isinstance ( results [ i ] , ABCSeries ) : 
 292 # If we have a view on v, we need to make a copy because 

 <command-1287881767488839> in <lambda> (x) 
 29 df = df [ df [ 'method_name' ] != 'search 2 nearbySearch' ] 
 30 
 ---> 31 function = lambda x : api_calls_parser ( ( apply_tt_api ( x , api , limit = limit , sleep = sleep ) ) , x . name , x [ 'searched_query' ] , threshold ) 
 32 
 33 result = df . apply ( function , axis = 1 ) 

 <command-924482031044831> in apply_tt_api (x, api_instance, limit, sleep, debug) 
 21 print ( f'collected searched_query: {searched_query}' ) 
 22 
 ---> 23 return api_instance . call_api ( { 'address' : searched_query , 'sleep' : sleep , 'limit' : limit , 'country' : country } , call_type = 'geocode' ) 

 /databricks/python/lib/python3.8/site-packages/decorator.py in fun (*args, **kw) 
 229 if not kwsyntax : 
 230 args , kw = fix ( args , kw , sig ) 
 --> 231 return caller ( func , * ( extras + args ) , ** kw ) 
 232 fun . __name__ = func . __name__
 233 fun . __doc__ = func . __doc__

 /databricks/python/lib/python3.8/site-packages/retry/api.py in retry_decorator (f, *fargs, **fkwargs) 
 71 args = fargs if fargs else list ( ) 
 72 kwargs = fkwargs if fkwargs else dict ( ) 
 ---> 73 return __retry_internal(partial(f, *args, **kwargs), exceptions, tries, delay, max_delay, backoff, jitter,
 74 logger)
 75 

 /databricks/python/lib/python3.8/site-packages/retry/api.py in __retry_internal (f, exceptions, tries, delay, max_delay, backoff, jitter, logger) 
 31 while _tries : 
 32 try : 
 ---> 33 return f ( ) 
 34 except exceptions as e : 
 35 _tries -= 1 

 <command-3472176617866502> in call_api (self, dict_params, call_type) 
 203 
 204 # Open the request 
 --> 205 with urllib . request . urlopen ( httpurl ) as url : 
 206 # Parses JSON from the URL 
 207 try : 

 /usr/lib/python3.8/urllib/request.py in urlopen (url, data, timeout, cafile, capath, cadefault, context) 
 220 else : 
 221 opener = _opener
 --> 222 return opener . open ( url , data , timeout ) 
 223 
 224 def install_opener ( opener ) : 

 /usr/lib/python3.8/urllib/request.py in open (self, fullurl, data, timeout) 
 529 for processor in self . process_res

In [ ]:
def full_function_make_api_calls_split(
    path_to_read: str, path_to_save: str, country_iso2: str, limit: int = 5, sleep: int = 1, top_split: bool = True, threshold: float = 0.85, test_size: int or None = None, seed: int = 101
) -> pd.DataFrame:
    """Function that makes the calls to the TomTom API after reading the Data from a path in the DBFS. You can pass an argument to save the api responses to another file in the DBFS as well.

    :param path_to_read: Path where you want to read the search logs from.
    :type path_to_read: str
    :param path_to_save: Path where you want to save the data, if any. Defaults to None, which means you don't need to save the results.
    :type path_to_save: str or None
    :param country_iso2: Country in ISO-2 format.
    :type country_iso2: str
    :param limit: Max number of responses you want from the TT API, defaults to 5
    :type limit: int, optional
    :param sleep: Time to wait between queries to the API server, defaults to 1
    :type sleep: int, optional
    :param threshold: The minimum level of confidence that is considered a match from the API response, defaults to 0.85, which means the confidence that the API has on what it is responding for the address to be a match is at least 85%.
    :type threshold: float, optional
    :param test_size: Size of the sample you want to take from the read dataframe. This parameter should only be used for testing purposes! It should be left at None! Defaults to None.
    :type test_size: int or None
    :return: A dataframe that contains the parsed response for each of the addresses returned. It contains the coordinates, the searched query, the confidence level, the type of query, the bounding box (if it is a geometry) and whether it is considered a match or not under the threshold criteria.
    :rtype: pd.DataFrame
    """
    api = TomtomApi()
    if ((test_size is None) or (test_size == 0)):
        df = pd.read_parquet(path_to_read).reset_index(drop=True)
    else:
        df = pd.read_parquet(path_to_read).reset_index(drop=True).sample(test_size, random_state=101)
    
    split_point = df.shape[0] // 2
    if top_split:
        df = df.sample(frac=1, random_state=seed)
        df = df.iloc[:split_point]
    else:
        df = df.sample(frac=1, random_state=seed)
        df = df.iloc[split_point:]
        
    df = df[df['method_name'] != 'search 2 nearbySearch']
    
    function = lambda x: api_calls_parser((apply_tt_api(x, api, limit=limit, sleep=sleep)), x.name, x['searched_query'], threshold)
    
    result =  df.apply(function, axis=1)
    
    final_df = pd.DataFrame(result.tolist())
    
    if path_to_save not in ['', None]:
        final_df.to_parquet(path_to_save)
        
    return final_df

In [ ]:
result_non_comp1 = full_function_make_api_calls_split(non_comp_path, non_comp_save_path1, country_iso2, 1, 1.5, top_split=True, threshold = 0.85)

In [ ]:
result_non_comp2 = full_function_make_api_calls_split(non_comp_path, non_comp_save_path2, country_iso2, 1, 1.5, top_split=True, threshold = 0.85)

In [ ]:
result_comp

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3167774330271509> in <module> 
 ----> 1 result_comp

 NameError : name 'result_comp' is not defined